In [18]:
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import pmdarima as pm


## Total renewable water resources per capita

In [6]:
# Read csv file for
df_1 = pd.read_csv('./dataset/AQUASTAT.csv')
# Filter to get columns with 'Total renewable water resources per capita'
df_1 = df_1[df_1['Variable'] == 'Total renewable water resources per capita']
# Get only columns with 'Area', 'Year', 'Value'
df_1 = df_1[['Area', 'Year', 'Value']]
# Rename columns 'Area', 'Variable'
df_1 = df_1.rename(columns={'Area': 'Country', 'Value': 'Total renewable water resources per capita (m3/inhab/year)'})
df_1.head()

,Country,Year,Total renewable water resources per capita (m3/inhab/year)
44877,Afghanistan,2015,1935.503042
44878,Afghanistan,2016,1886.176509
44879,Afghanistan,2017,1832.876970
44880,Afghanistan,2018,1780.750256
44881,Afghanistan,2019,1729.702584


## Water stress (%)

In [7]:
df_2 = pd.read_csv('./dataset/AQUASTAT_2.csv')
df_2 = df_2[df_2['Variable'] == 'SDG 6.4.2. Water Stress']
# Get only columns with 'Area', 'Year', 'Value'
df_2 = df_2[['Area', 'Year', 'Value']]
# Rename columns 'Area', 'Variable'
df_2 = df_2.rename(columns={'Area': 'Country', 'Value': 'Water Stress Index (%)'})

df_2.head()

,Country,Year,Water Stress Index (%)
7304,Afghanistan,2015,54.757019
7305,Afghanistan,2016,54.757019
7306,Afghanistan,2017,54.757019
7307,Afghanistan,2018,54.757019
7308,Afghanistan,2019,54.757019


## Population growth

In [8]:
df_3 = pd.read_csv('./dataset/Population.csv')
# Extrac certain columns
df_3 = df_3[['Country Name', '2015', '2016', '2017', '2018', '2019', '2020', '2021']]
# Create new columns called 'Value' and 'Year' and go through each row to get the value of each year 
df_3 = pd.melt(df_3, id_vars=['Country Name'], var_name='Year', value_name='Value')
# Convert 'Year' to integer
df_3['Year'] = df_3['Year'].astype(int)
# Reorder rows based on same 'Country Name' with ascending year
df_3 = df_3.sort_values(['Country Name', 'Year'])
# Rename columns 'Country Name', 'Value'
df_3 = df_3.rename(columns={'Country Name': 'Country', 'Value': 'Population (%)'})
df_3.head()

,Country,Year,Population (%)
2,Afghanistan,2015,3.121341
268,Afghanistan,2016,2.581549
534,Afghanistan,2017,2.866492
800,Afghanistan,2018,2.885208
1066,Afghanistan,2019,2.908529


## Combine dataset

In [9]:
# Group population growth, total freshwater withdrawal, total renewable water resources per capita by country and year
total_df = pd.merge(pd.merge(df_1, df_2, on=['Country', 'Year'], how='left'), df_3, on=['Country', 'Year'], how='left')
total_df.head()
# Remove rows with missing columns or 0 columns values
total_df = total_df.dropna()
# Save to csv file
total_df.to_csv('new_dataset.csv', index=False)
total_df.head()

,Country,Year,Total renewable water resources per capita (m3/inhab/year),Water Stress Index (%),Population (%)
0,Afghanistan,2015,1935.503042,54.757019,3.121341
1,Afghanistan,2016,1886.176509,54.757019,2.581549
2,Afghanistan,2017,1832.876970,54.757019,2.866492
3,Afghanistan,2018,1780.750256,54.757019,2.885208
4,Afghanistan,2019,1729.702584,54.757019,2.908529


In [19]:
# Go through df and get the unique values of 'Country'
countries = total_df['Country'].unique()
# Write the years and values of each country to a csv file
for country in countries:
    country_df = total_df[total_df['Country'] == country]
    country_df = country_df[['Year', 'Total renewable water resources per capita (m3/inhab/year)', 'Water Stress Index (%)', 'Population (%)']]
    country_df.to_csv(f'./dataset/{country}.csv', index=False)